In [88]:
import fastavro
import os

from mpire import WorkerPool

In [104]:
def process(resources: tuple[int, list[str]], worker_id: int) -> set[str]:
    
    n_workers, files = resources
    
    levels = set()
    
    for index, file in enumerate(files):
        if (index % n_workers == worker_id):
            with open(file, "rb") as f:
                reader = fastavro.reader(f)
                for sequence in reader:
                    inputs = sequence['merchant_city']
                    if inputs is not None:
                        levels.update(inputs)

    return levels

def multithread(n_workers: int, ):
    
    path = '/home/grantham/windmark/data/lifestreams/trxns'
    
    files = []
    
    for filename in os.listdir(path):
        if filename.endswith(".avro"):
            files.append(f"{path}/{filename}")
    
    resources: tuple[int, list[str]] = (n_workers, files)
    
    with WorkerPool(n_jobs=n_workers, shared_objects=resources) as pool:
        
        results: list[set[str]] = pool.map(process, range(n_workers))
        
        
    return results


    

In [112]:
results = stream(n_workers=1)

In [118]:
from functools import reduce

# sets
reduce( lambda a, b : a.union(b), results)

# counters
reduce( lambda a, b : a.update(b), results)

# tdigest
reduce( lambda a, b : a+b, results)

{' ONLINE',
 'Abrams',
 'Acton',
 'Afton',
 'Aiea',
 'Alameda',
 'Alamo',
 'Alamogordo',
 'Albuquerque',
 'Aledo',
 'Algiers',
 'Algodones',
 'Alhambra',
 'Aliso Viejo',
 'Alpine',
 'Alsip',
 'Alto',
 'Alvaton',
 'American Fork',
 'Amissville',
 'Amsterdam',
 'Anaheim',
 'Anchorage',
 'Anderson',
 'Angels Camp',
 'Ann Arbor',
 'Annapolis',
 'Antelope',
 'Antioch',
 'Apache Junction',
 'Apex',
 'Apple Valley',
 'Appleton',
 'Arlington',
 'Arlington Heights',
 'Arnold',
 'Asheboro',
 'Ashland',
 'Ashley',
 'Ashmore',
 'Athens',
 'Atkinson',
 'Atlanta',
 'Atlantic City',
 'Atmore',
 'Atwater',
 'Auburn',
 'Aurora',
 'Austin',
 'Avondale',
 'Babbitt',
 'Bailey',
 'Bakersfield',
 'Balch Springs',
 'Banning',
 'Barcelona',
 'Barneveld',
 'Batesville',
 'Bath',
 'Battle Creek',
 'Bay City',
 'Bayside',
 'Beacon',
 'Beaumont',
 'Beaverton',
 'Bedford',
 'Belding',
 'Bell Gardens',
 'Bellevue',
 'Bellflower',
 'Bend',
 'Bennett',
 'Bennettsville',
 'Benson',
 'Benton Harbor',
 'Berkeley',
 'Ber

In [113]:
import numpy as np

In [117]:
np.array([None, None], dtype=float)

array([nan, nan])

In [119]:
from collections import Counter

In [120]:
counter = Counter()

In [124]:
counter.update('hello')

AttributeError: 'Counter' object has no attribute 'add'

In [122]:
counter.update(['hello', 'world'])

In [123]:
counter

Counter({'l': 2, 'h': 1, 'e': 1, 'o': 1, 'hello': 1, 'world': 1})

In [1]:
from pathlib import Path

path = Path('/home/grantham/windmark/data/lifestreams/trxns')

from windmark.components.processors import multithread, counters, digests

In [2]:
results = multithread(n_workers=16, process=digests, key='amount', path=path)

In [8]:
from pytdigest import TDigest
import numpy as np

In [15]:

from functools import reduce

digests = [TDigest.of_centroids(np.array(digest)) for digest in results]

final = reduce( lambda a, b : a+b, digests)

In [16]:
final.force_merge()

In [17]:
final

TDigest(mean=37.5, weight=5.77e+06, centroids=43, merged, compression=100)